# Problem Satatement

# Sales Prediction for Big Mart Outlets
The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and predict the sales of each product at a particular outlet.

Using this model, BigMart will try to understand the properties of products and outlets which play a key role in increasing sales.

Please note that the data may have missing values as some stores might not report all the data due to technical glitches. Hence, it will be required to treat them accordingly. 



# Data Description
We have train (8523) and test (5681) data set, train data set has both input and output variable(s). You need to predict the sales for test data set.




CSV containing the item outlet information with sales value




* Variable	Description
* Item_Identifier	Unique product ID
* Item_Weight	Weight of product
* Item_Fat_Content	Whether the product is low fat or not
* Item_Visibility	The % of total display area of all products in a store allocated to the particular product
* Item_Type	The category to which the product belongs
* Item_MRP	Maximum Retail Price (list price) of the product
* Outlet_Identifier	Unique store ID
* Outlet_Establishment_Year	The year in which store was established
* Outlet_Size	The size of the store in terms of ground area covered
* Outlet_Location_Type	The type of city in which the store is located
* Outlet_Type	Whether the outlet is just a grocery store or some sort of supermarket
* Item_Outlet_Sales	Sales of the product in the particular store. This is the outcome variable to be predicted.

#1. Hypothesis Generation

**Store Level Hypotheses:**

1. City type: Stores located in urban or Tier 1 cities should have higher sales because of the higher income levels of people there.
2. Population Density: Stores located in densely populated areas should have higher sales because of more demand.
3. Store Capacity: Stores which are very big in size should have higher sales as they act like one-stop-shops and people would prefer getting everything from one place
4. Competitors: Stores having similar establishments nearby should have less sales because of more competition.
5. Marketing: Stores which have a good marketing division should have higher sales as it will be able to attract customers through the right offers and advertising.
6. Location: Stores located within popular marketplaces should have higher sales because of better access to customers.
7. Customer Behavior: Stores keeping the right set of products to meet the local needs of customers will have higher sales.
8. Ambiance: Stores which are well-maintained and managed by polite and humble people are expected to have higher footfall and thus higher sales.

**Product Level Hypotheses:**

1. Brand: Branded products should have higher sales because of higher trust in the customer.
2. Packaging: Products with good packaging can attract customers and sell more.
3. Utility: Daily use products should have a higher tendency to sell as compared to the specific use products.
4. Display Area: Products which are given bigger shelves in the store are likely to catch attention first and sell more.
5. Visibility in Store: The location of product in a store will impact sales. Ones which are right at entrance will catch the eye of customer first rather than the ones in back.
6. Advertising: Better advertising of products in the store will should higher sales in most cases.
7. Promotional Offers: Products accompanied with attractive offers and discounts will sell more.

#2. Data Exploration

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/ML_Projects/Regression/BigMart_Sales_prediction"

/content/drive/My Drive/Colab Notebooks/ML_Projects/Regression/BigMart_Sales_prediction


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [168]:
train['source'] = 'train'
test['source'] = 'test'
df = pd.concat([train, test],axis=0)

In [169]:
df.head(5)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


In [170]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14204 entries, 0 to 5680
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                11765 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                10188 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  source                     14204 non-null  object 
dtypes: float64(4), int64(1), object(8)
memory usage

In [171]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [172]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


* Item_Visibility has a min value of zero. This makes no practical sense because when a product is being sold in a store, the visibility cannot be 0.
* Outlet_Establishment_Years vary from 1985 to 2009. The values might not be apt in this form. Rather, if we can convert them to how old the particular store is, it should have a better impact on sales.
* The lower ‘count’ of Item_Weight and Item_Outlet_Sales confirms the findings from the missing value check.

In [173]:
df.nunique()

Item_Identifier               1559
Item_Weight                    415
Item_Fat_Content                 5
Item_Visibility              13006
Item_Type                       16
Item_MRP                      8052
Outlet_Identifier               10
Outlet_Establishment_Year        9
Outlet_Size                      3
Outlet_Location_Type             3
Outlet_Type                      4
Item_Outlet_Sales             3493
source                           2
dtype: int64

In [174]:
#Analysing Categorical variables
categorical_columns = [x for x in df.select_dtypes(include='object')]
categorical_columns = [x for x in categorical_columns if x not in ['Item_Identifier']]
for col in categorical_columns:
  print(f'\nFrequency of Categories for varible {col:s}:')
  print(df[col].value_counts())


Frequency of Categories for varible Item_Fat_Content:
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Frequency of Categories for varible Item_Type:
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Frequency of Categories for varible Outlet_Identifier:
OUT027    1559
OUT013    1553
OUT046    1550
OUT049    1550
OUT035    1550
OUT045    1548
OUT018    1546
OUT017    1543
OUT010     925
OUT019     880
Name: Outlet_Identifier, dtype: int64

Frequency of Categories f

* Item_Fat_Content: Some of ‘Low Fat’ values mis-coded as ‘low fat’ and ‘LF’. Also, some of ‘Regular’ are mentioned as ‘regular’.
* Item_Type: Not all categories have substantial numbers. It looks like combining them can give better results.
* Outlet_Type: Supermarket Type2 and Type3 can be combined. But we should check if that’s a good idea before doing it.

#3. Data Cleaning

inputing missing data: (`Item_Weight` and `Outlet_Size`)

In [175]:
df["Item_Weight"].isna().sum()

2439

In [176]:
Item_Weights = df.groupby('Item_Type')['Item_Weight'].mean().rename('Mean').to_frame()
Item_Weights['Max'] =  df.groupby('Item_Type')['Item_Weight'].max().rename('Max')
Item_Weights['Min'] =  df.groupby('Item_Type')['Item_Weight'].min().rename('Min')
Item_Weights['Count'] =  df.groupby('Item_Type')['Item_Weight'].count().rename('Count')

In [177]:
Item_Weights.sort_values(by='Mean',ascending=False)

,Mean,Max,Min,Count
Item_Type,,,,
Others,13.915724,20.50,5.500,228
Starchy Foods,13.659758,21.20,6.695,227
Household,13.337977,21.25,5.030,1285
Dairy,13.238358,20.70,4.805,941
Fruits and Vegetables,13.194406,21.35,5.460,1667
Breakfast,13.179112,21.10,6.425,152
Health and Hygiene,13.171739,21.25,5.175,710
Snack Foods,12.865732,21.25,5.095,1653
Meat,12.766104,21.25,5.150,607


In [178]:
df["Item_Weight"] = df.groupby("Item_Type")['Item_Weight'].transform(lambda x: x.fillna(x.mean()))

#Another way to do the same thing:
# df.loc[df['Item_Weight'].isna(),'Item_Weight'] = df.loc[df['Item_Weight'].isna(),'Item_Type'].apply(lambda x: Item_Weights['Mean'].loc[x])

In [179]:
df["Outlet_Size"].isna().sum()

4016

In [180]:
#Import mode function:
from scipy.stats import mode

In [181]:
df.groupby(['Outlet_Type','Outlet_Size'])['Outlet_Size'].count()

Outlet_Type        Outlet_Size
Grocery Store      Small           880
Supermarket Type1  High           1553
                   Medium         1550
                   Small          3100
Supermarket Type2  Medium         1546
Supermarket Type3  Medium         1559
Name: Outlet_Size, dtype: int64

In [182]:
df.pivot_table(index='Outlet_Type', values='Outlet_Size' ,aggfunc=(lambda x: mode(x)))

,Outlet_Size
Outlet_Type,
Grocery Store,"([Small], [880])"
Supermarket Type1,"([Small], [3100])"
Supermarket Type2,"([Medium], [1546])"
Supermarket Type3,"([Medium], [1559])"


In [183]:
outlet_size_mode = df.groupby("Outlet_Type")['Outlet_Size'].agg(lambda x: mode(x)[0])

In [184]:
miss_bool = df['Outlet_Size'].isnull() 

In [185]:
df.loc[miss_bool,'Outlet_Size'] = df.loc[miss_bool,'Outlet_Type'].apply(lambda x: outlet_size_mode[x])

#4. Feature Engineering

References:

- Analytics Vidhya - [Approach and Solution to break in Top 20 of Big Mart Sales prediction](https://www.analyticsvidhya.com/blog/2016/02/bigmart-sales-solution-top-20/)

- Kaggle - [Big Mart Sales Prediction Dataset](https://www.kaggle.com/devashish0507/big-mart-sales-prediction)